In [7]:
import requests
from bs4 import BeautifulSoup
from pyquery import PyQuery as pq
from datetime import datetime
import hashlib
import json
from pymongo import MongoClient
import time

## Config

In [ ]:
".sub-menu .sub-menu .menu-item-object-product_category a"

# Variables about mongodb
MONGODB_HOST = os.environ.get("MONGODB_HOST", "localhost")
MONGODB_PORT = int(os.environ.get("MONGODB_PORT", 27017))
MONGODB_USER = os.environ.get("MONGODB_USER", "")
MONGODB_PWD = os.environ.get("MONGODB_PWD", "")
DB = os.environ.get("DB", "yoooobot")

## Try

In [3]:
url_base = "http://www.timberland.com.tw"

In [9]:
res = requests.get(url_base)

In [13]:
doc = pq(res.text)
doc

[<html.no-js.zh-TW.tw>]

In [50]:
doc_categories = doc('.sub-menu .sub-menu .menu-item-object-product_category a')

In [51]:
categories = [(i.text(), i.attr('href')) for i in doc_categories.items()]
categories

[('T 恤', 'http://www.timberland.com.tw/men-apparel-t-shirts/'),
 ('運動上衣', 'http://www.timberland.com.tw/men-apparel-sweatshirts/'),
 ('POLO 衫', 'http://www.timberland.com.tw/men-apparel-polo/'),
 ('外套', 'http://www.timberland.com.tw/men-apparel-outerwear/'),
 ('襯衫', 'http://www.timberland.com.tw/men-apparel-shirts/'),
 ('褲款', 'http://www.timberland.com.tw/men-apparel-pants-shorts/'),
 ('毛衣', 'http://www.timberland.com.tw/men-apparel-sweaters/'),
 ('健行鞋／靴', 'http://www.timberland.com.tw/men-footwear-hiking-shoes-boots/'),
 ('靴款', 'http://www.timberland.com.tw/men-footwear-boots/'),
 ('運動靴', 'http://www.timberland.com.tw/men-footwear-sneakerboots/'),
 ('帆船鞋', 'http://www.timberland.com.tw/men-footwear-boat-shoes/'),
 ('休閒鞋款', 'http://www.timberland.com.tw/men-footwear-shoes/'),
 ('涼鞋', 'http://www.timberland.com.tw/men-footwear-sandals/'),
 ('T 恤', 'http://www.timberland.com.tw/women-apparel-t-shirts/'),
 ('運動上衣', 'http://www.timberland.com.tw/women-apparel-sweatshirts/'),
 ('洋裝', 'http:

## Refactor

**Records schema**

```json
{
    'product_number': 'A1LELL69',
    'product_name': '男士Squam Lake 彈性直筒牛仔褲',
    'color': '穿舊感洗色',
    'cate1': '男士',
    'cate2': '男裝',
    'cate3': '褲款'
    'price': 3900,
    'img_src': 'http://xxx.xxx/123.png',
    'desc': '具舒適彈性的包芯纱牛仔布 大樹標誌 零錢口袋上緣鉚釘裝飾－正面零錢口袋與後口袋均採四針車縫線設計樣式 內側褲頭下方縫線與 Timberland 靴款顏色鞋帶相應',
    'features': None,
    'product_url': 'http://xx.xx'
}
```

In [54]:
def get_product(url, session=None):
    pass
    # return list_of_producs

In [55]:
def _get_single_page_prodects(url, session=None):
    pass

In [ ]:
def get_product_list(cate_url, session=None):
    pass

In [57]:
def get_categogies(url, session=None):
    pass

In [ ]:
def get_products()